# Q2. Fine-tuning a Pre-trained ResNet50 on iNaturalist (Subset)
This notebook loads a pre-trained ResNet50 model from ImageNet, adapts it for the 10-class iNaturalist subset, and fine-tunes the model using two strategies with two-phases: first freezing the backbone, then unfreezing other layers.

In [ ]:
!wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip
!unzip -q nature_12K.zip

In [ ]:
!pip install -q torch torchvision

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import os

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

In [ ]:
# Transforms
IMG_SIZE = 224
BATCH_SIZE = 32
imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std  = [0.229, 0.224, 0.225]
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=imagenet_mean, std=imagenet_std),
])
val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=imagenet_mean, std=imagenet_std),
])

In [ ]:
# Dataset
data_dir = '/content/inaturalist_12K'
train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=train_transform)
val_dataset = datasets.ImageFolder(os.path.join(data_dir, 'val'), transform=val_transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
num_classes = len(train_dataset.classes)

In [ ]:
# Load pre-trained model and modify final layer
model = models.resnet50(pretrained=True)
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, num_classes)
model = model.to(device)

### Strategy 1: Fine Tuning Entire Model

In [ ]:
# Training utilities
criterion = nn.CrossEntropyLoss()
def train_one_epoch(model, loader, optimizer):
    model.train()
    total_loss, total_correct = 0, 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * images.size(0)
        total_correct += (outputs.argmax(1) == labels).sum().item()
    return total_loss / len(loader.dataset), total_correct / len(loader.dataset)

def validate(model, loader):
    model.eval()
    total_loss, total_correct = 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * images.size(0)
            total_correct += (outputs.argmax(1) == labels).sum().item()
    return total_loss / len(loader.dataset), total_correct / len(loader.dataset)

In [ ]:
# Stage 1: Train only classifier layer
for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters():
    param.requires_grad = True

optimizer = torch.optim.Adam(model.fc.parameters(), lr=1e-3)
for epoch in range(3):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer)
    val_loss, val_acc = validate(model, val_loader)
    print(f'[Stage 1 - Epoch {epoch+1}] Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}')

In [ ]:
# Stage 2: Fine-tune entire model
for param in model.parameters():
    param.requires_grad = True
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
for epoch in range(5):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer)
    val_loss, val_acc = validate(model, val_loader)
    print(f'[Stage 2 - Epoch {epoch+1}] Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}')

## Strategy 2

In [ ]:
import os, time

In [ ]:
# Freeze all layers first
for param in model.parameters():
    param.requires_grad = False

# Unfreeze layer4 and fc
for name, param in model.named_parameters():
    if "layer4" in name or "fc" in name:
        param.requires_grad = True

In [ ]:
# Training logic with logging
criterion = nn.CrossEntropyLoss()

def train_one_epoch(model, loader, optimizer, epoch=0, log_every=20):
    model.train()
    total_loss, total_correct = 0, 0
    start_time = time.time()
    for batch_idx, (images, labels) in enumerate(loader):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * images.size(0)
        total_correct += (outputs.argmax(1) == labels).sum().item()
        if (batch_idx + 1) % log_every == 0:
            print(f"[Epoch {epoch+1} | Batch {batch_idx+1}/{len(loader)}] Loss: {loss.item():.4f}")
    avg_loss = total_loss / len(loader.dataset)
    accuracy = total_correct / len(loader.dataset)
    print(f"[Epoch {epoch+1}] Train Loss: {avg_loss:.4f}, Train Acc: {accuracy:.4f}, Time: {time.time()-start_time:.2f}s")
    return avg_loss, accuracy

def validate(model, loader, epoch=0):
    model.eval()
    total_loss, total_correct = 0, 0
    start_time = time.time()
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * images.size(0)
            total_correct += (outputs.argmax(1) == labels).sum().item()
    avg_loss = total_loss / len(loader.dataset)
    accuracy = total_correct / len(loader.dataset)
    print(f"[Validation] Val Loss: {avg_loss:.4f}, Val Acc: {accuracy:.4f}, Time: {time.time()-start_time:.2f}s")
    return avg_loss, accuracy


In [ ]:
# Phase 1: Train only layer4
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)
for epoch in range(3):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, epoch=epoch)
    val_loss, val_acc = validate(model, val_loader, epoch=epoch)

In [ ]:
# Phase 2: Unfreeze all and fine-tune further
for param in model.parameters():
    param.requires_grad = True

optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
for epoch in range(3, 6):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, epoch=epoch)
    val_loss, val_acc = validate(model, val_loader, epoch=epoch)